In [817]:
import pandas as pd
import math
from iapws import IAPWS97
import sys

%run conservation_equations_v2.ipynb

sys.stdout = open("log.txt", "w")

In [818]:
class node:
     def __init__(self, nodeID, P, mf, z):
        self.nodeID = nodeID
        self.P = P
        self.mf = mf
        self.z = z
        
class pipeline:
     def __init__(self, lineID, length, in_dia, roughness, pipe_t, insul_t, pipe_k, insul_k, n_elbow45, n_elbow90, n_tee, \
                 n_rbend, n_coupling, n_union, n_gatefull, n_gatehalf, out_dia):
        self.length = length
        self.in_dia = in_dia 
        self.roughness = roughness
        self.pipe_t = pipe_t 
        self.insul_t = insul_t 
        self.pipe_k = pipe_k
        self.insul_k = insul_k
        self.n_elbow45 = n_elbow45
        self.n_elbow90 = n_elbow90
        self.n_tee = n_tee
        self.n_rbend = n_rbend
        self.n_coupling = n_coupling
        self.n_union = n_union
        self.n_gatefull = n_gatefull
        self.n_gatehalf = n_gatehalf
        self.out_dia = out_dia     #for pipe expansion or contraction

In [819]:
node_df = pd.read_excel("bom.xls", sheetname=8, parse_cols="A:D")
pipeline_df = pd.read_excel("bom.xls", sheetname=7)
#print (pipeline_df.head())

node_dict = {}
pipeline_dict = {}

for nodes in range(len(node_df.index)):
    #create a node class for each row and store it in a dictionary
    nodeID, P, mf, z = map(lambda x: node_df.iloc[nodes, x], range(0,4))
    node_dict[nodeID] = node(nodeID, P, mf, z)

for pipelines in range(len(pipeline_df.index)):
    #create a pipeline class for each row and store it in a dictionary
    lineID, length, in_dia, roughness, pipe_t, insul_t, pipe_k, insul_k, n_elbow45, n_elbow90, n_tee, \
        n_rbend, n_coupling, n_union, n_gatefull, n_gatehalf, out_dia = \
        map(lambda x: pipeline_df.iloc[pipelines, x], range(0,17))   
    pipeline_dict[lineID] = pipeline(lineID, length, in_dia, roughness, pipe_t, insul_t, pipe_k, insul_k, \
        n_elbow45, n_elbow90, n_tee, n_rbend, n_coupling, n_union, n_gatefull, n_gatehalf, out_dia)

#print(node_dict.keys())
#print(pipeline_dict.keys())

In [820]:
def pressure_drop(z1, z2, Diameter, MassFlow, InletPressure, Length, Elbow_90, GateValve_FullyOpen, \
                  ThermalConductivity, InsulationThickness, Diameter2):
    '''Calculate parameters from steam table'''
    sat_steam = IAPWS97(P = InletPressure, x=1)
    temperature = sat_steam.T
    density = sat_steam.rho
    enthalpy =  sat_steam.h
    dynamic_viscosity = sat_steam.mu
    #print(temperature, density, dynamic_viscosity, enthalpy)
    
    '''Calculate Derived Parameters'''
    Diameter = Diameter
    #Relative roughness
    roughness = 0.000046
    relative_roughness = roughness / Diameter
    #cross sectional area
    area = math.pi * Diameter**2 / 4
    area2 = math.pi * Diameter2**2 / 4
    #velocity
    velocity = MassFlow / (density * area)
    velocity2 = MassFlow/ (density * area2)
    #print(velocity, velocity2)
    
    '''Skin Friction Calculation'''
    nRE = reynolds_number(Diameter, velocity, density, dynamic_viscosity)
    friction_factor = ff_colebrook(nRE, relative_roughness)
    friction_skin = f_skin(temperature, Diameter, velocity, relative_roughness, Length)
    print('sking friction :', friction_skin, velocity, area, MassFlow )
    
    '''Pipe Enlargement and contraction '''
    if area == area2:
        friction_expansion = 0
        friction_contraction = 0
    else:
        if area < area2:
            friction_expansion = f_expansion(area, area2, velocity)
            friction_contraction = 0
            #print('expansion')
        else:
            friction_expansion = 0
            friction_contraction = f_contraction(area, area2, velocity)
            #print('contraction')
    
    """Define Fittings"""
    #Represent in a list of 8
    fittings_list = [0, 0, 0, 0, 0, 0, 0, 0]
    fittings_list[1] = Elbow_90
    fittings_list[6] = GateValve_FullyOpen
    #fittings [0] = elbow_45  #fittings [1] = elbow_90
    #fittings [2] = tee       #fittings [3] = return_bend
    #fittings [4] = coupling  #fittings [5] = union
    #fittings [6] = gate_valve_full  #fittings [7] = gate_valve_half
    '''Losses in fittings and valves'''
    friction_fittings = f_fittings(velocity, fittings_list)
    #print (velocity, friction_fittings)
    
    '''Calulate total friction losses'''
    friction_total = friction_skin + friction_expansion + friction_contraction + friction_fittings
    
    
    '''Define Parameters for heat loss'''
    #Convective heat transfer coefficient (h; W/m**K)
    h_steam_pipe = 36
    h_insulation_air = 7
    #Insulation
    t_insulation = InsulationThickness          #insulation thickness (m)
    k_insulation = ThermalConductivity          #insulation's thermal conductivity (W/m** K)
    #Pipe
    t_pipe = 0.5 * 0.0254          #pipeline thickness (m)
    k_pipe = 56           #pipe's thermal conductivity (W/m** K)
    #id_pipe = 0.25        #pipe's inner diameter (m) given at diameter
    #l_pipe = 6000         #length of pipeline (m) given at length
    #Temp
    t1 = temperature - 273.15           #temp inside the pipeline
    t2 = 25                             #temp; atmospheric
    
    r1 = Diameter
    r2 = Diameter + t_pipe
    r3 = Diameter + t_pipe + t_insulation
    
    h_t = heat_transfer(Length, t1, t2, h_steam_pipe, h_insulation_air, k_pipe, k_insulation, r1, r2, r3)
    Q = h_t / MassFlow
    
    '''Calculate gdZ'''
    gdz = head(z1, z2)
    
    '''Calculate udu'''
    udu = (velocity**2 - velocity2**2)/2

    '''Calculate outlet pressure'''
    p2 = p_out(InletPressure, friction_total, gdz, udu, density, Q )
    t_loss = friction_total - Q + gdz + udu
    
    '''Calculate steam flow at the end of pipeline'''
    X = x_out(t_loss*-1e-3, InletPressure, p2)
    sf_out = X*MassFlow
    
    print('{:26}'.format('Velocity (m/s)'), '{:>16.2f}'.format(velocity))
    print('{:26}'.format('Tot Loss (J/kg)'), '{:>16.2f}'.format(t_loss))
    print('{:26}'.format('Heat Loss (J/kg)'), '{:>16.2f}'.format(-Q))
    print ('{:26}'.format('d K.E. (J/kg)'), '{:>16.2f}'.format(udu))
    print ('{:26}'.format('d P.E. (J/kg)'), '{:>16.2f}'.format(gdz))
    print ('{:26}'.format('Friction Loss (J/kg)'), '{:>16.2f}'.format(friction_total))
    print ('{:26}'.format('  Skin Friction (J/kg)'), '{:>16.2f}'.format(friction_skin))
    print ('{:26}'.format('  Expansion (J/kg)'), '{:>16.2f}'.format(friction_expansion))
    print ('{:26}'.format('  Contraction (J/kg)'), '{:>16.2f}'.format(friction_contraction))
    print ('{:26}'.format('  Fittings Friction (J/kg)'), '{:>16.2f}'.format(friction_fittings))
    print ('{:26}'.format('Inlet Pressure (Mpa)'), '{:>16.2f}'.format(InletPressure))
    print ('{:26}'.format('Outlet Pressure (Mpa)'), '{:>16.2f}'.format(p2))
    print('{:26}'.format('SF_in (kg/s)'), '{:>16.2f}'.format(MassFlow))
    print('{:26}'.format('SF_out (kg/s)'), '{:>16.2f}'.format(sf_out))
    
    return [p2, sf_out]

In [821]:
#given nodes 504 and 701 and STL504
#calc dp along STL 504
#print (node_dict['701A'].z)
#edge = {'STL504':[504, '701A']}
#
#t_loss = pressure_drop(node_dict[504].z, node_dict['701A'].z, pipeline_dict['STL504'].in_dia, node_dict[504].mf, \
#                       node_dict[504].P, pipeline_dict['STL504'].length, pipeline_dict['STL504'].n_elbow90, \
#                       pipeline_dict['STL504'].n_gatefull, pipeline_dict['STL504'].insul_k, \
#                       pipeline_dict['STL504'].insul_t, pipeline_dict['STL504'].out_dia)

In [822]:
edge = {'STL501':[501, '702A'], 'STL502':[502, '702A'], 'STL503':[503, '702A'], 'STL504':[504, '701A'], \
        'STL505':[505, '701A'], 'STL506':[506, '701B'], 'STL508':[508, '702C'], 'STL702A':['702A', '702B'], \
        'STL702B':['702B', '702C'], 'STL702IF':['702C', '702IF'], 'STL701A':['701A', '701B'], \
        'STL701B':['701B', '701C'], 'STL701IF':['701C', '701IF']}

#print (sorted(edge.keys())
node_out={}

for x in sorted(edge):
    #edge[x][0] for input node; edge[x][1] for output node; x for pipeline
    print('SteamLine', x, edge[x])
    #print (node_dict[edge[x][0]].z)
    t_loss = pressure_drop(node_dict[edge[x][0]].z, node_dict[edge[x][1]].z, pipeline_dict[x].in_dia, \
                       node_dict[edge[x][0]].mf, \
                       node_dict[edge[x][0]].P, pipeline_dict[x].length, pipeline_dict[x].n_elbow90, \
                       pipeline_dict[x].n_gatefull, pipeline_dict[x].insul_k, \
                       pipeline_dict[x].insul_t, pipeline_dict[x].out_dia)
    
    if node_dict[edge[x][1]].P == 0:
        node_dict[edge[x][1]].P = t_loss[0]
    else:
        node_dict[edge[x][1]].P = (node_dict[edge[x][1]].P + t_loss[0] )/2
    node_dict[edge[x][1]].mf += t_loss[1]
    print(pipeline_dict[x].length, node_dict[edge[x][1]].mf)
    
    
    #print(t_loss)
    ##store output node result in a dictionary
    #if edge[x][1] not in node_out.keys():
    #    node_out[edge[x][1]]=t_loss
    #    print(node_out[edge[x][1]])
    #else:
    #    node_out[edge[x][1]][0] = (node_out[edge[x][1]][0] + t_loss[0])/2
    #    node_out[edge[x][1]][1] = node_out[edge[x][1]][1] + t_loss[1]
    #    print(node_out[edge[x][1]])
    #print(node_out.keys())    
        
    print ('\n', '===================================================', '\n')

In [823]:

print(sorted(edge.keys()))